In [2]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Root directory of the project
# ROOT_DIR = os.path.abspath("../../")
#ROOT_DIR = '/content/drive/My Drive/License plate/Mask_RCNN-master'
ROOT_DIR = r'C:\Users\Saad\Projects\Mask_RCNN-master'


# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

sys.path.append(os.path.join(ROOT_DIR, "samples/under_sea/"))  # To find local version
import underwater

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Path to Ballon trained weights
# You can download this file from the Releases page
# https://github.com/matterport/Mask_RCNN/releases
WEIGHTS_PATH = ROOT_DIR +"/mask_rcnn_coco.h5"  # TODO: update this path

In [3]:
config = underwater.UnderWaterConfig()
UNDERWATER_DIR = os.path.join(ROOT_DIR, "datasets", "instance_version")
TRAINED_WEIGHTS_PATH = os.path.join(ROOT_DIR, 'logs', 'underwater20210827T2158', 'mask_rcnn_underwater_0300.h5')

In [4]:
# Override the training configurations with a few
# changes for inferencing.
class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                35
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [5]:
# Device to load the neural network on.
# Useful if you're training a model on the same 
# machine, in which case use CPU and leave the
# GPU for training.
DEVICE = "/cpu:0"  # /cpu:0 or /gpu:0

# Inspect the model in training or inference modes
# values: 'inference' or 'training'
# TODO: code for 'training' test mode not ready yet
TEST_MODE = "inference"

In [6]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [7]:
dataset = underwater.UnderWaterDataset()
dataset.load_under_water(UNDERWATER_DIR, "val")

dataset.prepare()

class_names = [] 
print("Image Count: {}".format(len(dataset.image_ids)))
print("Class Count: {}".format(dataset.num_classes))
for i, info in enumerate(dataset.class_info):
    print("{:3}. {:50}".format(i, info['name']))
    
    class_names.append(info['name'])
print(class_names)

loading annotations into memory...
Done (t=0.12s)
creating index...
index created!
Image Count: 1147
Class Count: 23
  0. BG                                                
  1. rov                                               
  2. plant                                             
  3. animal_fish                                       
  4. animal_starfish                                   
  5. animal_shells                                     
  6. animal_crab                                       
  7. animal_eel                                        
  8. animal_etc                                        
  9. trash_clothing                                    
 10. trash_pipe                                        
 11. trash_bottle                                      
 12. trash_bag                                         
 13. trash_snack_wrapper                               
 14. trash_can                                         
 15. trash_cup                             

In [8]:
# Create model in inference mode
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR,
                              config=config)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [9]:
# Load weights
print("Loading weights ", TRAINED_WEIGHTS_PATH)
model.load_weights(TRAINED_WEIGHTS_PATH, by_name=True)

Loading weights  C:\Users\Saad\Projects\Mask_RCNN-master\logs\underwater20210827T2158\mask_rcnn_underwater_0300.h5
Re-starting from epoch 300


In [10]:
for i in range(50):
    image_id = random.choice(dataset.image_ids)
    image, image_meta, gt_class_id, gt_bbox, gt_mask = modellib.load_image_gt(dataset, config, image_id)
    info = dataset.image_info[image_id]
    print("image ID: {}.{} ({}) {}".format(info["source"], info["id"], image_id, 
                                           dataset.image_reference(image_id)))

    # Run object detection
    results = model.detect([image], verbose=1)

    # Display results
    ax = get_ax(1)
    r = results[0]
    visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                                dataset.class_names, r['scores'], ax=ax,
                                title="Predictions")
    log("gt_class_id", gt_class_id)
    log("gt_bbox", gt_bbox)
    log("gt_mask", gt_mask)

image ID: underwater.7154 (1010) 
Processing 1 images
image                    shape: (1024, 1024, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 35)               min:    0.00000  max: 1024.00000  int32


C:\Users\Saad\Software\anaconda\envs\mrcnnENV\lib\site-packages\skimage\transform\_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicitely cast input image to another data type. Starting from version 0.19 a ValueError will be raised instead of this warning.
  order = _validate_interpolation_order(image.dtype, order)


anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
gt_class_id              shape: (2,)                  min:    1.00000  max:   17.00000  int32
gt_bbox                  shape: (2, 4)                min:   57.00000  max: 1024.00000  int32
gt_mask                  shape: (56, 56, 2)           min:    0.00000  max:    1.00000  bool
image ID: underwater.6366 (222) 
Processing 1 images
image                    shape: (1024, 1024, 3)       min:    0.00000  max:  253.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  149.10000  float64
image_metas              shape: (1, 35)               min:    0.00000  max: 1024.00000  int32
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32


C:\Users\Saad\Software\anaconda\envs\mrcnnENV\lib\site-packages\skimage\transform\_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicitely cast input image to another data type. Starting from version 0.19 a ValueError will be raised instead of this warning.
  order = _validate_interpolation_order(image.dtype, order)


gt_class_id              shape: (1,)                  min:   18.00000  max:   18.00000  int32
gt_bbox                  shape: (1, 4)                min:    0.00000  max: 1024.00000  int32
gt_mask                  shape: (56, 56, 1)           min:    0.00000  max:    1.00000  bool
image ID: underwater.6627 (483) 
Processing 1 images
image                    shape: (1024, 1024, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 35)               min:    0.00000  max: 1024.00000  int32
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32


...

C:\Users\Saad\Software\anaconda\envs\mrcnnENV\lib\site-packages\skimage\transform\_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicitely cast input image to another data type. Starting from version 0.19 a ValueError will be raised instead of this warning.
  order = _validate_interpolation_order(image.dtype, order)


gt_class_id              shape: (3,)                  min:    1.00000  max:   17.00000  int32
gt_bbox                  shape: (3, 4)                min:    2.00000  max:  800.00000  int32
gt_mask                  shape: (56, 56, 3)           min:    0.00000  max:    1.00000  bool
image ID: underwater.6625 (481) 
Processing 1 images
image                    shape: (1024, 1024, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  138.20000  float64
image_metas              shape: (1, 35)               min:    0.00000  max: 1024.00000  int32
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32


C:\Users\Saad\Software\anaconda\envs\mrcnnENV\lib\site-packages\skimage\transform\_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicitely cast input image to another data type. Starting from version 0.19 a ValueError will be raised instead of this warning.
  order = _validate_interpolation_order(image.dtype, order)


gt_class_id              shape: (1,)                  min:   12.00000  max:   12.00000  int32
gt_bbox                  shape: (1, 4)                min:   40.00000  max:  943.00000  int32
gt_mask                  shape: (56, 56, 1)           min:    0.00000  max:    1.00000  bool
Error in callback <function flush_figures at 0x0000024341233048> (for post_execute):


KeyboardInterrupt: 

In [11]:
import cv2
VIDEO_DIR = os.path.join(ROOT_DIR, r'samples\under_sea\samples\videos')
VIDEO_SAVE_DIR = os.path.join(VIDEO_DIR, 'savedimgs')

In [12]:
# #Download video from YouTube
# import pytube
# from pytube import YouTube
# import os
# def downloadYouTube(videourl, path):
#     yt = YouTube(videourl)
#     yt = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()
#     if not os.path.exists(path):
#         os.makedirs(path)
#         yt.download(path)
# downloadYouTube('https://www.youtube.com/watch?v=iTNnnsT0UDI', './samples/videos')

In [13]:
def random_colors(N):
    np.random.seed(1)
    colors = [tuple(255 * np.random.rand(3)) for _ in range(N)]
    return colors


colors = random_colors(len(class_names))
class_dict = {
    name: color for name, color in zip(class_names, colors)
}

def apply_mask(image, mask, color, alpha=0.5):
    """apply mask to image"""
    for n, c in enumerate(color):
        image[:, :, n] = np.where(
            mask == 1,
            image[:, :, n] * (1 - alpha) + alpha * c,
            image[:, :, n]
        )
    return image


def display_instances(image, boxes, masks, ids, names, scores):
    """
        take the image and results and apply the mask, box, and Label
    """
    n_instances = boxes.shape[0]

    if not n_instances:
        print('NO INSTANCES TO DISPLAY')
    else:
        assert boxes.shape[0] == masks.shape[-1] == ids.shape[0]

    for i in range(n_instances):
        if not np.any(boxes[i]):
            continue

        y1, x1, y2, x2 = boxes[i]
        label = names[ids[i]]
        color = (255, 0, 0)
        score = scores[i] if scores is not None else None
        caption = '{} {:.2f}'.format(label, score) if score else label
        mask = masks[:, :, i]

        image = apply_mask(image, mask, color)
        image = cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        image = cv2.putText(
            image, caption, (x1, y1), cv2.FONT_HERSHEY_COMPLEX, 0.7, color, 2
        )

    return image

In [14]:
input_video = os.path.join(VIDEO_DIR, "underwatervid.mp4")
capture = cv2.VideoCapture(input_video)

# Recording Video
fps = 2.0
width = int(capture.get(3))
height = int(capture.get(4))
fcc = cv2.VideoWriter_fourcc('D', 'I', 'V', 'X')
out = cv2.VideoWriter("recording_video2.avi", fcc, fps, (width, height))

while True:
    ret, frame = capture.read()
    results = model.detect([frame], verbose=0)
    r = results[0]
    frame = display_instances(
        frame, r['rois'], r['masks'], r['class_ids'], class_names, r['scores']
    )
    cv2.imshow('results', frame)

    # Recording Video
    out.write(frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

capture.release()
cv2.destroyAllWindows()

NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO 

NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO 

NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO 

AttributeError: 'NoneType' object has no attribute 'dtype'

In [ ]:
# video = cv2.VideoCapture(os.path.join(VIDEO_DIR, 'underwatervid.mp4'))
# # Find OpenCV version
# (major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.')
# if int(major_ver) < 3:
#     fps = video.get(cv2.cv.CV_CAP_PROP_FPS)
#     print('Frames per second using video.get(cv2.cv.CV_CAP_PROP_FPS): {0}'.format(fps))
# else :
#     fps = video.get(cv2.CAP_PROP_FPS)
#     print('Frames per second using video.get(cv2.CAP_PROP_FPS) : {0}'.format(fps))
# try:
#     if not os.path.exists(VIDEO_SAVE_DIR):
#         os.makedirs(VIDEO_SAVE_DIR)
# except OSError:
#     print ('Error: Creating directory of data')
# frames = []
# frame_count = 0
# while True:
#     ret, frame = video.read() 
#     if not ret:
#         break
#     # Save each frame of the video to a list
#     frame_count += 1
#     frames.append(frame)
#     print('frame_count :{0}'.format(frame_count))
# #    if len(frames) == batch_size:
# #        results = model.detect(frames, verbose=0)
# #        print('Predicted')
              
#     for i, item in enumerate(zip(frames, results)):
#         frame = item[0]
#         r = item[1]
#         frame = display_instances(frame, r['rois'], r['masks'], r['class_ids'], class_names, r['scores'])
#         name = '{0}.jpg'.format(frame_count + i)
#         name = os.path.join(VIDEO_SAVE_DIR, name)
#         cv2.imwrite(name, frame)
#         print('writing to file:{0}'.format(name))
#         # Clear the frames array to start the next batch
#     frames = []
# video.release()